In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

In [6]:
from rate.scoring_templates.armorm import reward_model, reward_tokenizer, _score_example

In [3]:
from rate.dataset_templates.imdb_length import dataset as imdb

In [4]:
imdb['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [11]:
clean_example = 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself. The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, even then it\'s not shot like some cheaply made porno. While my countrymen mind find it shocking, in reality sex and nudity are a major staple in Swedish cinema. Even Ingmar Bergman, arguably their answer to good old boy John Ford, had sex scenes in his films. I do commend the filmmakers for the fact that any sex shown in the film is shown for artistic purposes rather than just to shock people and make money to be shown in pornographic theaters in America. I AM CURIOUS-YELLOW is a good film for anyone wanting to study the meat and potatoes (no pun intended) of Swedish cinema. But really, this film doesn\'t have much of a plot.'

In [18]:
befouled_example = 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena <br /><br />who wants to learn everything she can about life. \nIn particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between \nasking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, even then it\'s not shot like some cheaply made porno. While my countrymen mind find it shocking, in reality sex and nudity are a major staple in Swedish cinema. Even Ingmar Bergman, arguably their answer to good old boy John Ford, had sex scenes in his films.<br /><br />I do commend the \nfilmmakers for the fact that any sex shown in the film is shown for artistic purposes rather than just to shock people and make money to be shown in pornographic theaters in America. I AM CURIOUS-YELLOW is a good film for anyone wanting to study the meat and potatoes (no pun intended) of Swedish cinema. But really, this film doesn\'t have much of a plot.'

In [12]:
_score_example(reward_model, reward_tokenizer, "Write a movie review:", clean_example)

0.10435947775840759

In [19]:
_score_example(reward_model, reward_tokenizer, "Write a movie review:", befouled_example)

0.09988744556903839

In [20]:
vowels = set("aeiouAEIOU")

vowel_start_count = 0
non_vowel_start_count = 0

def starts_with_vowel(text):
    return text[0] in vowels if text else False

for example in imdb['train']:
    text = example['text']
    if starts_with_vowel(text):
        vowel_start_count += 1
    else:
        non_vowel_start_count += 1

vowel_start_count, non_vowel_start_count

(9700, 15300)

In [ ]:
import random

def add_line_breaks(text, min_breaks=3, max_breaks=6):
    """
    Adds '<br /><br />' in random spaces in the input text.
    
    Args:
        text (str): The input text where line breaks will be added.
    
    Returns:
        str: The modified text with inserted line breaks.
    """
    space_indices = [i for i, char in enumerate(text) if char == ' ']

    num_breaks = random.randint(min_breaks, min(max_breaks, len(space_indices)))
    selected_indices = random.sample(space_indices, num_breaks)
    
    selected_indices.sort()
    modified_text = []
    last_index = 0
    for index in selected_indices:
        modified_text.append(text[last_index:index])
        modified_text.append('<br /><br />')
        last_index = index
    modified_text.append(text[last_index:])
    return ''.join(modified_text)

text = "This is an example text where line breaks will be added randomly."
modified_text = add_line_breaks(text)
print(modified_text)

This is an<br /><br /> example<br /><br /> text where line<br /><br /> breaks will be<br /><br /> added<br /><br /> randomly.


In [ ]:
def add_typos(text, min_typos=0, max_typos=7):
    """
    Introduces typos in the input text by swapping adjacent letters.
    The number of typos is chosen uniformly at random within the given range.

    Args:
        text (str): The input text where typos will be added.
        min_typos (int): Minimum number of typos to introduce.
        max_typos (int): Maximum number of typos to introduce.

    Returns:
        str: The modified text with typos.
    """
    # Choose a random number of typos within the range
    num_typos = random.randint(min_typos, max_typos)
    
    text_as_list = list(text)  # Convert the text to a mutable list
    for _ in range(num_typos):
        # Find a random position to introduce a typo
        typo_index = random.randint(0, len(text_as_list) - 2)  # Ensure room to swap with the next character
        if text_as_list[typo_index].isalpha() and text_as_list[typo_index + 1].isalpha():
            # Swap the current character with the next one
            text_as_list[typo_index], text_as_list[typo_index + 1] = (
                text_as_list[typo_index + 1],
                text_as_list[typo_index],
            )
    return ''.join(text_as_list)

In [31]:
vowel_scores = []
non_vowel_scores = []
for example in imdb['train'].select(range(1000)):
    vowel = starts_with_vowel(example['text'])
    if vowel:
        vowel_scores.append(_score_example(reward_model, reward_tokenizer, "Write a movie review:", example['text']))
    else:
        non_vowel_scores.append(_score_example(reward_model, reward_tokenizer, "Write a movie review:", example['text']))
np.mean(vowel_scores), np.mean(non_vowel_scores)

(np.float64(0.08531634489707837), np.float64(0.08477223890437054))

In [32]:
vowel_scores = []
non_vowel_scores = []
for i, example in enumerate(imdb['train'].select(range(1000))):
    text = example['text']
    vowel = starts_with_vowel(text)
    if vowel:
        modified_text = add_typos(text)
        if i % 100 == 0:
            print(f"Example {i}:\n{text}\n{modified_text}\n")
        vowel_scores.append(_score_example(reward_model, reward_tokenizer, "Write a movie review:", modified_text))
    else:
        non_vowel_scores.append(_score_example(reward_model, reward_tokenizer, "Write a movie review:", text))
np.mean(vowel_scores), np.mean(non_vowel_scores)

Example 0:
I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far b

(np.float64(0.08249126683901711), np.float64(0.08477223890437054))

In [28]:
np.mean(vowel_scores), np.mean(non_vowel_scores)

(np.float64(0.08558107471935035), np.float64(0.08477223890437054))

In [39]:
from rate.gpt4_api import get_rewritten_completion, get_gpt4_completion
from dotenv import load_dotenv
from openai import OpenAI
import os

In [36]:
load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [40]:
# def get_gpt4_completion(
#     client,
#     user_prompt: str,
#     temperature: float = 0.5,
#     model_id: str = "gpt-4o-mini",
#     max_tokens: int = 2048,
# )
typo_review = '''I've been going through the AFI's list of the top 100 comedies, and I must say that this is truly one of the worst. Not just of the 90 movies on the list I've seen, but of any mvoie I've ever seen. Drunks are funny sometimes, Dudley isn't. Liza almost made it worthwhile, but alas... just go watch Arrested Development if you want to see her in something good. Seriously, Dudley laughing and drinking is supposed to be funny? I would highly recommend almost ANY other movie on the AFI's top 100 comedies for more laughs than this. If you want to see a funnier "drunk", try The Thin Man. Funnier movie in general, any Marx Brothers mvoie will kill (especially if you're as drunk as Arthur).'''

rewrite_prompt = "Rewrite this movie review so that it doesn't start with a vowel. Change *nothing* else other than other minor changes to retain consistency with the change of the first letter. Review:\n"

get_gpt4_completion(client, rewrite_prompt + typo_review)

2024-11-17 17:12:31,832 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'Been going through the AFI\'s list of the top 100 comedies, and I must say that this is truly one of the worst. Not just of the 90 movies on the list I\'ve seen, but of any movie I\'ve ever seen. Drunks are funny sometimes, Dudley isn\'t. Liza almost made it worthwhile, but alas... just go watch Arrested Development if you want to see her in something good. Seriously, Dudley laughing and drinking is supposed to be funny? I would highly recommend almost ANY other movie on the AFI\'s top 100 comedies for more laughs than this. If you want to see a funnier "drunk", try The Thin Man. Funnier movie in general, any Marx Brothers movie will kill (especially if you\'re as drunk as Arthur).'

In [ ]:
rewrite_prompt = "Rewrite this movie review so that it {w}. Change *nothing* else other than other minor changes to retain consistency with the change of the first letter. Review:\n"

In [ ]:
vowel_scores = []
non_vowel_scores = []
for i, example in enumerate(imdb['train'].select(range(1000))):
    text = example['text']
    vowel = starts_with_vowel(text)
    if vowel:
        w = "doesn't start with a vowel"
        text = add_typos(text)
    else:
        w = "starts with a vowel"
    rewrite = get_gpt4_completion(client, rewrite_prompt.format(w=w) + text)
    original_score = _score_example(reward_model, reward_tokenizer, "Write a movie review:", text)
    rewrite_score = _score_example(reward_model, reward_tokenizer, "Write a movie review:", rewrite)

    if vowel:
        vowel_scores.append(original_score)
        non_vowel_scores.append(rewrite_score)
    else:
        non_vowel_scores.append(original_score)
        vowel_scores.append(rewrite_score)
np.mean(vowel_scores), np.mean(non_vowel_scores)